# library

In [4]:
!pip install tensorflow --q


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# installing 3 libraries in Command prompt:
# pip install pandas scikit-learn tensorflow
#
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Data 


 load some of your data libraries, and here I have given an example of a dictionary

In [ ]:
# example of data
#
data = {
    'Recipe': [
        'Pasta Carbonara', 'Greek Salad', 'Fried Chicken',
        'Spaghetti Bolognese', 'Vegetarian Tacos',
        'Stuffed Cabbage', 'Stuffed Peppers', 'Moussaka', 'Cheese Pie', 'Fish Soup'
    ],
    'Ingredients': [
        'pasta, eggs, pancetta, parmesan, pepper',
        'tomato, cucumber, feta, olives, red onion, oregano',
        'chicken, flour, eggs, oil, salt, pepper',
        'spaghetti, ground meat, tomato sauce, onion, garlic, parmesan',
        'tortillas, beans, corn, avocado, tomato, red onion, cheese',
        'sauerkraut, ground meat, rice, onion, spices',
        'peppers, ground meat, rice, tomato sauce, spices',
        'potatoes, ground meat, eggs, milk, spices',
        'phyllo dough, cheese, eggs, oil',
        'river fish, vegetables, spices, tomato, garlic'
    ],
    'Cuisine': [
        'Italian', 'Greek', 'American',
        'Italian', 'Mexican',
        'Eastern European', 'Eastern European', 'Greek', 'Greek', 'Eastern European'
    ],
    'Preparation Time (mins)': [
        20, 15, 30, 25, 20, 45, 40, 60, 50, 35
    ],
    'Calories': [
        400, 200, 500, 450, 300, 350, 400, 600, 450, 300
    ]
}

# Augmenting the data with additional recipes and ingredients
additional_data = {
    'Recipe': [
        'Chicken Curry', 'Beef Stew', 'Vegetable Stir Fry',
        'Lentil Soup', 'Fish Tacos', 'Quiche Lorraine',
        'Caesar Salad', 'Chocolate Cake', 'Apple Pie', 'Garlic Bread'
    ],
    'Ingredients': [
        'chicken, curry powder, coconut milk, onion, garlic, ginger',
        'beef, potatoes, carrots, onions, garlic, beef broth',
        'broccoli, bell peppers, soy sauce, garlic, ginger, tofu',
        'lentils, carrots, celery, onion, garlic, vegetable broth',
        'fish, tortillas, cabbage, lime, avocado, salsa',
        'eggs, cream, bacon, cheese, pie crust, onion',
        'romaine lettuce, croutons, parmesan, caesar dressing, chicken',
        'flour, sugar, cocoa powder, eggs, butter, baking powder',
        'apples, sugar, cinnamon, pie crust, butter, lemon juice',
        'bread, garlic, butter, parsley, parmesan'
    ],
    'Cuisine': [
        'Indian', 'American', 'Asian',
        'Middle Eastern', 'Mexican', 'French',
        'American', 'American', 'American', 'Italian'
    ],
    'Preparation Time (mins)': [
        40, 120, 25, 30, 20, 45, 15, 60, 50, 15
    ],
    'Calories': [
        600, 700, 250, 300, 350, 400, 200, 500, 450, 150
    ]
}

# Adding the additional data to the existing data
for key in data:
    data[key].extend(additional_data[key])

# create data fram structure
#
df = pd.DataFrame(data)
#
# show dataframe

In [104]:
df.head()

,Recipe,Ingredients,Cuisine,Preparation Time (mins),Calories
0,Pasta Carbonara,"pasta, eggs, pancetta, parmesan, pepper",Italian,20,400
1,Greek Salad,"tomato, cucumber, feta, olives, red onion, ore...",Greek,15,200
2,Fried Chicken,"chicken, flour, eggs, oil, salt, pepper",American,30,500
3,Spaghetti Bolognese,"spaghetti, ground meat, tomato sauce, onion, g...",Italian,25,450
4,Vegetarian Tacos,"tortillas, beans, corn, avocado, tomato, red o...",Mexican,20,300


In [105]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

# Preprocess Ingredients
tfidf_ingredients = TfidfVectorizer(tokenizer=lambda x: x.split(', '), lowercase=False)
X = tfidf_ingredients.fit_transform(df['Ingredients']).toarray()

# Preprocess Cuisine and Recipe
label_encoder_cuisine = LabelEncoder()
y_cuisine = label_encoder_cuisine.fit_transform(df['Cuisine'])

label_encoder_recipe = LabelEncoder()
y_recipe = label_encoder_recipe.fit_transform(df['Recipe'])

# Prepare numerical targets
y_time = df['Preparation Time (mins)']
y_calories = df['Calories']

# Split the data
X_train, X_val, y_train_cuisine, y_val_cuisine, y_train_recipe, y_val_recipe, y_train_time, y_val_time, y_train_calories, y_val_calories = train_test_split(
    X, y_cuisine, y_recipe, y_time, y_calories, test_size=0.2, random_state=42)

# Define the model
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer = Dense(128, activation='relu')(input_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

# Output layers
output_cuisine = Dense(len(label_encoder_cuisine.classes_), activation='softmax', name='cuisine')(hidden_layer)
output_recipe = Dense(len(label_encoder_recipe.classes_), activation='softmax', name='recipe')(hidden_layer)
output_time = Dense(1, activation='linear', name='time')(hidden_layer)
output_calories = Dense(1, activation='linear', name='calories')(hidden_layer)

# Compile the model
model = Model(inputs=input_layer, outputs=[output_cuisine, output_recipe, output_time, output_calories])
model.compile(optimizer='adam', loss={'cuisine': 'sparse_categorical_crossentropy', 
                                      'recipe': 'sparse_categorical_crossentropy', 
                                      'time': 'mean_squared_error', 
                                      'calories': 'mean_squared_error'})

# Train the model
model.fit(X_train, {'cuisine': y_train_cuisine, 'recipe': y_train_recipe, 'time': y_train_time, 'calories': y_train_calories},
          validation_data=(X_val, {'cuisine': y_val_cuisine, 'recipe': y_val_recipe, 'time': y_val_time, 'calories': y_val_calories}),
          epochs=50, batch_size=32)

# Make predictions on validation set
predictions = model.predict(X_val)

# Extract predictions
predicted_cuisine_indices = predictions[0].argmax(axis=1)
predicted_cuisine = label_encoder_cuisine.inverse_transform(predicted_cuisine_indices)
predicted_recipe_indices = predictions[1].argmax(axis=1)
predicted_recipe = label_encoder_recipe.inverse_transform(predicted_recipe_indices)
predicted_time = predictions[2]
predicted_calories = predictions[3]

# Print the results for validation set
for i in range(len(X_val)):
    print(f"Ingredients: {df.iloc[i]['Ingredients']}")
    print(f"Predicted Cuisine: {predicted_cuisine[i]}")
    print(f"Predicted Recipe: {predicted_recipe[i]}")
    print(f"Predicted Preparation Time: {predicted_time[i][0]:.2f} mins")
    print(f"Predicted Calories: {predicted_calories[i][0]:.2f} kcal")
    print("-" * 40)

Epoch 1/50


d:\azdin\.env\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - calories_loss: 179270.8750 - cuisine_loss: 2.1812 - loss: 181385.7500 - recipe_loss: 3.0123 - time_loss: 2109.6731 - val_calories_loss: 152540.3281 - val_cuisine_loss: 2.1270 - val_loss: 154104.6094 - val_recipe_loss: 2.9681 - val_time_loss: 1559.1865
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - calories_loss: 179275.6719 - cuisine_loss: 2.1882 - loss: 181387.7656 - recipe_loss: 3.0085 - time_loss: 2106.8882 - val_calories_loss: 152525.4219 - val_cuisine_loss: 2.1253 - val_loss: 154089.3281 - val_recipe_loss: 2.9726 - val_time_loss: 1558.8125
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - calories_loss: 179243.0625 - cuisine_loss: 2.1786 - loss: 181355.9219 - recipe_loss: 3.0355 - time_loss: 2107.6621 - val_calories_loss: 152510.6250 - val_cuisine_loss: 2.1237 - val_loss: 154074.1875 - val_recipe_loss: 2.9769 - val_time_loss: 1558.4514
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - calories_loss: 179211.1562 - cuisine_loss: 2.1989 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense



# Preprocess Ingredients
tfidf_ingredients = TfidfVectorizer(tokenizer=lambda x: x.split(', '), lowercase=False)
X = tfidf_ingredients.fit_transform(df['Ingredients']).toarray()

# Preprocess Cuisine and Recipe
label_encoder_cuisine = LabelEncoder()
y_cuisine = label_encoder_cuisine.fit_transform(df['Cuisine'])

label_encoder_recipe = LabelEncoder()
y_recipe = label_encoder_recipe.fit_transform(df['Recipe'])

# Prepare numerical targets
y_time = df['Preparation Time (mins)']
y_calories = df['Calories']

# Split the data
X_train, X_val, y_train_cuisine, y_val_cuisine, y_train_recipe, y_val_recipe, y_train_time, y_val_time, y_train_calories, y_val_calories = train_test_split(
    X, y_cuisine, y_recipe, y_time, y_calories, test_size=0.2, random_state=42)

# Define the model
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer = Dense(128, activation='relu')(input_layer)

# Output layers
output_cuisine = Dense(len(label_encoder_cuisine.classes_), activation='softmax', name='cuisine')(hidden_layer)
output_recipe = Dense(len(label_encoder_recipe.classes_), activation='softmax', name='recipe')(hidden_layer)
output_time = Dense(1, activation='linear', name='time')(hidden_layer)
output_calories = Dense(1, activation='linear', name='calories')(hidden_layer)

# Compile the model
model = Model(inputs=input_layer, outputs=[output_cuisine, output_recipe, output_time, output_calories])
model.compile(optimizer='adam', loss={'cuisine': 'sparse_categorical_crossentropy', 
                                      'recipe': 'sparse_categorical_crossentropy', 
                                      'time': 'mean_squared_error', 
                                      'calories': 'mean_squared_error'})

# Train the model
model.fit(X_train, {'cuisine': y_train_cuisine, 'recipe': y_train_recipe, 'time': y_train_time, 'calories': y_train_calories},
          validation_data=(X_val, {'cuisine': y_val_cuisine, 'recipe': y_val_recipe, 'time': y_val_time, 'calories': y_val_calories}),
          epochs=50, batch_size=32)

# Make predictions on validation set
predictions = model.predict(X_val)

# Extract predictions
predicted_cuisine_indices = predictions[0].argmax(axis=1)
predicted_cuisine = label_encoder_cuisine.inverse_transform(predicted_cuisine_indices)
predicted_recipe_indices = predictions[1].argmax(axis=1)
predicted_recipe = label_encoder_recipe.inverse_transform(predicted_recipe_indices)
predicted_time = predictions[2]
predicted_calories = predictions[3]

# Print the results for validation set
for i in range(len(X_val)):
    print(f"Ingredients: {df.iloc[i]['Ingredients']}")
    print(f"Predicted Cuisine: {predicted_cuisine[i]}")
    print(f"Predicted Recipe: {predicted_recipe[i]}")
    print(f"Predicted Preparation Time: {predicted_time[i][0]:.2f} mins")
    print(f"Predicted Calories: {predicted_calories[i][0]:.2f} kcal")
    print("-" * 40)

# Example ingredients prediction
example_ingredients = [
    ['pasta', 'eggs', 'pancetta', 'parmesan', 'pepper'],
    ['tomato', 'cucumber', 'feta', 'olives', 'red onion', 'oregano'],
    ['chicken', 'flour', 'eggs', 'oil', 'salt', 'pepper'],
    ['spaghetti', 'ground meat', 'tomato sauce', 'onion', 'garlic'],
    ['tortillas', 'beans', 'corn', 'avocado', 'tomato', 'red onion']
]

# Convert example ingredients to strings
example_ingredients_str = [', '.join(ingredients) for ingredients in example_ingredients]

# Vectorize the example ingredients
example_vectors = tfidf_ingredients.transform(example_ingredients_str).toarray()

# Make predictions on example ingredients
predictions_example = model.predict(example_vectors)

# Extract predictions
predicted_cuisine_indices_example = predictions_example[0].argmax(axis=1)
predicted_cuisine_example = label_encoder_cuisine.inverse_transform(predicted_cuisine_indices_example)
predicted_recipe_indices_example = predictions_example[1].argmax(axis=1)
predicted_recipe_example = label_encoder_recipe.inverse_transform(predicted_recipe_indices_example)
predicted_time_example = predictions_example[2]
predicted_calories_example = predictions_example[3]

# Print the results for example ingredients
for i in range(len(example_ingredients)):
    print(f"Ingredients: {example_ingredients[i]}")
    print(f"Predicted Cuisine: {predicted_cuisine_example[i]}")
    print(f"Predicted Recipe: {predicted_recipe_example[i]}")
    print(f"Predicted Preparation Time: {predicted_time_example[i][0]:.2f} mins")
    print(f"Predicted Calories: {predicted_calories_example[i][0]:.2f} kcal")
    print("-" * 40)


In [109]:
# save the models
import joblib

# Define the tokenizer function separately
def custom_tokenizer(text):
    return text.split(', ')

# Assign the custom tokenizer to the TfidfVectorizer
tfidf_ingredients.tokenizer = custom_tokenizer

model.save('models2/recipe_model.h5')
joblib.dump(tfidf_ingredients, 'models2/tfidf_ingredients.pkl')
joblib.dump(label_encoder_cuisine, 'models2/label_encoder_cuisine.pkl')
joblib.dump(label_encoder_recipe, 'models2/label_encoder_recipe.pkl')


# load the models
from tensorflow.keras.models import load_model
import pickle

model = load_model('models2/recipe_model.h5')
tfidf_ingredients = joblib.load('models2/tfidf_ingredients.pkl')
label_encoder_cuisine = joblib.load('models2/label_encoder_cuisine.pkl')
label_encoder_recipe = joblib.load('models2/label_encoder_recipe.pkl')


# Example ingredients prediction
example_ingredients = [
    ['pasta', 'eggs', 'pancetta', 'parmesan', 'pepper'],
    ['tomato', 'cucumber', 'feta', 'olives', 'red onion', 'oregano'],
    ['chicken', 'flour', 'eggs', 'oil', 'salt', 'pepper'],
    ['spaghetti', 'ground meat', 'tomato sauce', 'onion', 'garlic'],
    ['tortillas', 'beans', 'corn', 'avocado', 'tomato', 'red onion']
]

# Convert example ingredients to strings
example_ingredients_str = [', '.join(ingredients) for ingredients in example_ingredients]

# Vectorize the example ingredients
example_vectors = tfidf_ingredients.transform(example_ingredients_str).toarray()

# Make predictions on example ingredients
predictions_example = model.predict(example_vectors)

# Extract predictions
predicted_cuisine_indices_example = predictions_example[0].argmax(axis=1)

predicted_cuisine_example = label_encoder_cuisine.inverse_transform(predicted_cuisine_indices_example)
predicted_recipe_indices_example = predictions_example[1].argmax(axis=1)
predicted_recipe_example = label_encoder_recipe.inverse_transform(predicted_recipe_indices_example)
predicted_time_example = predictions_example[2]
predicted_calories_example = predictions_example[3]

# Print the results for example ingredients
for i in range(len(example_ingredients)):
    print(f"Ingredients: {example_ingredients[i]}")
    print(f"Predicted Cuisine: {predicted_cuisine_example[i]}")
    print(f"Predicted Recipe: {predicted_recipe_example[i]}")
    print(f"Predicted Preparation Time: {predicted_time_example[i][0]:.2f} mins")
    print(f"Predicted Calories: {predicted_calories_example[i][0]:.2f} kcal")
    print("-" * 40)
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Ingredients: ['pasta', 'eggs', 'pancetta', 'parmesan', 'pepper']
Predicted Cuisine: Italian
Predicted Recipe: Beef Stew
Predicted Preparation Time: 0.75 mins
Predicted Calories: 1.03 kcal
----------------------------------------
Ingredients: ['tomato', 'cucumber', 'feta', 'olives', 'red onion', 'oregano']
Predicted Cuisine: American
Predicted Recipe: Fish Soup
Predicted Preparation Time: 0.32 mins
Predicted Calories: 0.70 kcal
----------------------------------------
Ingredients: ['chicken', 'flour', 'eggs', 'oil', 'salt', 'pepper']
Predicted Cuisine: American
Predicted Recipe: Fried Chicken
Predicted Preparation Time: 0.73 mins
Predicted Calories: 1.50 kcal
----------------------------------------
Ingredients: ['spaghetti', 'ground meat', 'tomato sauce', 'onion', 'garlic']
Predicted Cuisine: American
Predicted Recipe: Fried Chicken
Predicted Preparation Time: 0.80 mins
Predicted Calories: 1.89 kcal
----------------------------------------
Ingredi

# Conversions and coding of ingredient data

One should convert the ingredients into TF-IDF functions using TfidfVectorizer(). TfidfVectorizer is a tool from the scikit-learn library that converts text data into numeric functions using the TF-IDF method. TF-IDF (Term Frequency-Inverse Document Frequency) is a method for evaluating the importance of words in a set of documents. If you have ingredients such as "pasta, eggs, pancetta", TfidfVectorizer will create a feature vector for each recipe where each component in the vector represents a TF-IDF value for a specific word (ingredient). For example:

If you have ingredients such as "pasta, eggs, pancetta", TfidfVectorizer will create a feature vector for each recipe where each component in the vector represents a TF-IDF value for a specific word (ingredient). For example:

Recipes:

1. pasta, eggs, pancetta
2. tomato, cucumber, feta

List:

['pasta', 'eggs', 'pancetta', 'tomato', 'cucumber', 'feta']

TF-IDF vectors (simplified) represent the relative importance of the ingredients in each recipe:

Recept 1: [0.5, 0.5, 0.5, 0, 0, 0]

Recept 2: [0, 0, 0, 0.5, 0.5, 0.5]

In [ ]:
# Convert ingredients (text data) to TF-IDF numeric functions:
#
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Sastojci']).toarray()
#
# Create instances of LabelEncoder()
label_encoder = LabelEncoder()
#
# # Encoding a recipe as a label (creating a label)
y = label_encoder.fit_transform(df['Recept'])
#
# Converting 'y' labels (encoding) to a one-hot vector
y = to_categorical(y)

# Division into training and test sets

The model should be taught familiar recipes and ingredients so that he can choose easily later
recipe as an output classifier in relation to the input data, in this case it is the ingredients

In [ ]:
# Split the data into train and test sets
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Machine learning model

In [ ]:
# A deep neural network model
#
# Creating a sequential model
model = Sequential()
#
# 1. hidden layer with dimensioned input
model.add(Dense(256, input_dim=X.shape[1], activation='relu'))
# Dropout layer against network overfitting
model.add(Dropout(0.2))
#
# 2. hidden layer

model.add(Dense(128, activation='relu'))
# Dropout layer against network overfitting
model.add(Dropout(0.2))
#
# 3. hidden layer
model.add(Dense(64, activation='relu'))
# Dropout layer against network overfitting
model.add(Dropout(0.2))
#
# The output dense layer of the network
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
#
# Compiling the model
#
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

d:\azdin\.env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training model

In [11]:
model.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.0000e+00 - loss: 2.2981 - val_accuracy: 0.0000e+00 - val_loss: 2.3184
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0868 - loss: 2.2843 - val_accuracy: 0.0000e+00 - val_loss: 2.3247
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.2986 - loss: 2.2329 - val_accuracy: 0.0000e+00 - val_loss: 2.3349
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.2986 - loss: 2.2631 - val_accuracy: 0.0000e+00 - val_loss: 2.3470
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3854 - loss: 2.2159 - val_accuracy: 0.0000e+00 - val_loss: 2.3592
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0868 - loss: 2.2344 - val_accuracy: 0.0000e+00 - val_loss: 2.3696
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1736 - loss: 2.1810 - val_accuracy: 0.0000e+00 - val_loss: 2.3824
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5347 - loss: 2.1534 - val_acc

# Recipe recommendation based on input ingredients

In [ ]:
# Recipe recommendation function
#
def preporuci_recept(sastojci):
    sastojci_vec = vectorizer.transform([sastojci]).toarray()
    predikcija = model.predict(sastojci_vec)
    indeks = predikcija.argmax()
    return label_encoder.inverse_transform([indeks])[0]

# model testing - example

In [ ]:
# example of usage
#
kupljene_namirnice = 'tortilje, pasulj, kukuruz, avokado, paradajz, crveni luk, sir'
preporuceni_recept = preporuci_recept(kupljene_namirnice)
print(f'Preporučeni recept: \n{preporuceni_recept}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Preporučeni recept: 
Vegetarijanski Tacos


In [16]:
import joblib
from tensorflow.keras.models import save_model
import os

# Create the directory if it does not exist
os.makedirs('models', exist_ok=True)

# Save the label encoder
joblib.dump(label_encoder, 'models/label_encoder.pkl')

# Save the prediction model
save_model(model, 'models/recipe_prediction_model.h5')

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'models/tfidf_vectorizer.pkl')

['models/tfidf_vectorizer.pkl']

In [17]:
import joblib
from tensorflow.keras.models import load_model

# Load the label encoder
label_encoder = joblib.load('models/label_encoder.pkl')

# Load the prediction model
model = load_model('models/recipe_prediction_model.h5')

# Load the TF-IDF vectorizer
vectorizer = joblib.load('models/tfidf_vectorizer.pkl')
# Test the model on an example input
example_input = 'pasta, jaja, panceta, parmezan, biber'
predicted_recept = preporuci_recept(example_input)
print(f'Predicted Recept for example input: {predicted_recept}')


# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Predicted Recept for example input: Pasta Carbonara
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step - accuracy: 0.0000e+00 - loss: 3.0077
Test Loss: 3.0076889991760254
Test Accuracy: 0.0
